# Paper to Audio

This notebook will take in an HTML file of a paper and parse it into an audio book.
TTS is generated through Kokoro.

In [1]:
from bs4 import BeautifulSoup
import re
from typing import List, Dict

def clean_text(text: str) -> str:
    """Clean text by removing non-ASCII characters and multiple spaces."""
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return ' '.join(text.split())

def process_paragraph(p_tag) -> str:
    """Process a paragraph tag by removing unwanted elements and cleaning text."""
    # Remove math and cite tags
    for tag in p_tag.find_all(['math', 'cite']):
        tag.decompose()
    
    text = clean_text(p_tag.get_text(strip=False))
    return text

def process_abstract(soup) -> Dict[str, str]:
    """Extract and process the abstract section."""
    abstract = soup.find('div', class_='ltx_abstract')
    if not abstract:
        return None
    
    abstract_paras = []
    for p in abstract.find_all('p', class_='ltx_p'):
        text = process_paragraph(p)
        if text:
            abstract_paras.append(text)
    
    return {
        "title": "Abstract",
        "content": "\n".join(abstract_paras)
    }

def process_section(section) -> Dict[str, str]:
    """Process a single section and return its title and content."""
    # Extract title
    title_tag = section.find('h2', class_='ltx_title_section')
    title = title_tag.get_text(strip=True) if title_tag else "No Title"
    title = re.sub(r'^\d+', '', title)
    
    # Process paragraphs
    paragraphs = []
    for p in section.find_all('p', class_='ltx_p'):
        text = process_paragraph(p)
        if text:
            paragraphs.append(text)
    
    return {
        "title": title.strip(),
        "content": "\n".join(paragraphs)
    }

def parse_paper_html(file_path: str) -> List[Dict[str, str]]:
    """Main function to parse the paper HTML and return structured content."""
    # Read and parse HTML
    with open(file_path, "r") as f:
        html = f.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    # Initialize result list
    section_list = []
    
    # Process abstract
    abstract_section = process_abstract(soup)
    if abstract_section:
        section_list.append(abstract_section)
    
    # Process main sections
    sections = soup.find_all('section', class_='ltx_section')
    for section in sections:
        section_data = process_section(section)
        section_list.append(section_data)
    
    return section_list

file_path = "/home/iris/wsl_shared/paper_audio_book_generation/src_data/DeepSeek-V3 Technical Report.html"
section_list = parse_paper_html(file_path)

# # Print sections for verification
# for section in section_list:
#     print(f"\n=== {section['title']} ===")
#     print(f"Content length: {len(section['content'])} characters")


In [ ]:
section_list[3]["content"]

In [ ]:
from kokoro.models import build_model
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL = build_model('kokoro/kokoro-v0_19.pth', device)
VOICE_NAME = [
    'af', # Default voice is a 50-50 mix of Bella & Sarah
    'af_bella', 'af_sarah', 'am_adam', 'am_michael',
    'bf_emma', 'bf_isabella', 'bm_george', 'bm_lewis',
    'af_nicole', 'af_sky',
][0]
VOICEPACK = torch.load(f'kokoro/voices/{VOICE_NAME}.pt', weights_only=True).to(device)
print(f'Loaded voice: {VOICE_NAME}')

In [ ]:
from kokoro.kokoro import generate
import nltk
import numpy as np
import soundfile as sf
from tqdm import tqdm
# Download the punkt tokenizer if you haven't already
nltk.download('punkt')

def text_to_audio(text, model, voicepack, voice_name):
    """Generate audio for a single piece of text"""
    audio, _ = generate(model, text, voicepack, lang=voice_name[0])
    return audio

def process_sections(section_list, model, voicepack, voice_name):
    # Initialize empty list to store audio arrays
    audio_segments = []
    
    for section in section_list:
        print(f"Procssing Section title: {section['title']}")
        # Generate audio for the title
        title_audio = text_to_audio(section["title"], model, voicepack, voice_name)
        audio_segments.append(title_audio)
        
        # Add a short pause after the title
        silence = np.zeros(12000)  # 1 second silence at 24kHz
        audio_segments.append(silence)
        
        # Split content into sentences
        sentences = nltk.sent_tokenize(section["content"])
        
        # Generate audio for each sentence
        for sentence in tqdm(sentences):
            # print(sentence)
            try:
                sentence_audio = text_to_audio(sentence, model, voicepack, voice_name)
            except:
                # Split sentence by commas
                parts = sentence.split(',')
                parts = [part.strip() for part in parts if part.strip()]  # Remove empty parts and whitespace
                
                # Initialize array for combined audio
                audio_parts = []
                
                # Process each part separately
                for part in parts:
                    part_audio = text_to_audio(part, model, voicepack, voice_name)
                    audio_parts.append(part_audio)
                
                # Concatenate all parts
                sentence_audio = np.concatenate(audio_parts)
            audio_segments.append(sentence_audio)

            
            # Add a short pause between sentences
            silence = np.zeros(12000)  # 0.5 second silence
            audio_segments.append(silence)
            
        # Add a longer pause between sections
        section_silence = np.zeros(24000)  # 2 seconds silence
        audio_segments.append(section_silence)
    
    # Concatenate all audio segments
    combined_audio = np.concatenate(audio_segments)
    return combined_audio

# Main execution
def generate_audiobook(section_list, model, voicepack, voice_name, output_path):
    # Process all sections and get combined audio
    final_audio = process_sections(section_list, model, voicepack, voice_name)
    
    # Save as WAV file first
    wav_path = output_path.replace('.mp3','.wav')
    sf.write(wav_path, final_audio, 24000)
    
    # Convert to MP3 using pydub
    from pydub import AudioSegment
    audio = AudioSegment.from_wav(wav_path)
    audio.export(output_path, format="mp3")
    
    # Remove temporary WAV file
    import os
    os.remove(wav_path)
    print(f"Audio book has been saved to {output_path}")
    return final_audio

# Usage example:

# Assuming you have MODEL, VOICEPACK, and VOICE_NAME defined as in your notebook
output_file = "audiobook2.mp3"
final_audio = generate_audiobook(section_list, MODEL, VOICEPACK, VOICE_NAME, output_file)



In [ ]:
# from IPython.display import display, Audio

# display(Audio(data=final_audio, rate=24000, autoplay=True))

### Transform to MP4 for YouTube

In [ ]:
from moviepy.editor import ImageClip, AudioFileClip, CompositeVideoClip
import numpy as np

image_clip = ImageClip("src_data/DeepSeek-V3TechnicalReport.png")
audio_clip = AudioFileClip("DeepSeek-V3 Technical Report.mp3")
video = image_clip.set_audio(audio_clip)
video = video.set_duration(audio_clip.duration)
video.write_videofile("DeepSeek-V3_Technical_Report.mp4", fps=1)